## A* (A Star) Method

https://leetcode.com/problems/shortest-path-in-a-grid-with-obstacles-elimination/solution/ に詳しい。

```
A* というこの一風変わった名前は、この論文でスタートからゴールまでの最短経路を確実に見つけるアルゴリズムを許容的 (英: admissible) と呼び、論文の数式中に 許容的なアルゴリズムの集合を A と表し、そのAの中でも評価回数が最適になる物を A* と表記していたためである[7]。
```

ヒューリスティック関数がadmissive(ゴールまでのコストを、実際にかかるコストよりも低く見積もることはない)らしい。
この場合、最初(の回?)に見つかった経路が最短経路であることが保証される(なんでだ)。
(注意)最初とはいえ、その時の回のqueueの要素は全部見たうえで決める必要がある。

---

```
A*の考え方
スタートノードから、あるノード n を通って、ゴールノードまでたどり着くときの最短経路を考える。このときこの最短経路のコストを f* (n) とおくと、

f* (n)= g* (n) + h* (n)

と置くことが出来る。ここで g* (n) はスタートノードから n までの最小コスト、h* (n) はn からゴールノードまでの最小コストである。もし g* (n) の値と h* (n)の値を知っていれば、全体の最短経路f* (n) は容易に求まる。しかしながら実際には g* (n) と h* (n) をあらかじめ与えることは出来ない。そこで f* (n) を次のような推定値 f (n) に置き換えることを考える。

f(n) = g(n) + h(n)

ここで g(n) はスタートノードから n までの最小コストの推定値、h(n) は n からゴールノードまでの最小コストの推定値である。この場合 g に関しては探索の過程で更新を加えることによりg*に近づけてゆくことができるが、 h* は、実際にゴールに辿り着くまでは誰にもわからない。そこで、 h(n) には人間が(ある程度妥当性を持つように)設計した推定値を与えることにする。このアルゴリズムを A*探索アルゴリズムといい、 h (n) をヒューリスティック関数という。
```

---

```
最良優先探索（best-first search）は、幅優先探索（英: breadth-first search）を何らかの規則（評価関数）に従って次に探索する最も望ましいノードを選択するように拡張した探索アルゴリズムである。

探索ノードを効率的に選択するには優先度付きキュー（英: priority queue）を用いて実装するのが一般的である。キューに貯めずに最良のノードだけを扱うと山登り法になる。キューを評価関数でソートしないと幅優先探索になる。

最良優先探索の例としてはダイクストラ法（英: Dijkstra's algorithm）やA*アルゴリズム（英: A* search algorithm）や均一コスト探索を挙げることができる。最良優先探索は経路探索においてしばしば使われるアルゴリズムである。コンピュータ将棋・コンピュータチェスなどでも最良優先探索を拡張した物が使われている。
```

http://kussharo.complex.eng.hokudai.ac.jp/~kurihara/classes/AI/heuristic.pdf がめっちゃわかりやすく、均一コスト探索(＝ダイクストラ？)、貪欲最良優先探索、A*メソッドを説明している。


なおh(n) = 0の場合はダイクストラ法に他ならない。    
というか、ダイクストラ法の探索順を最適化してるみたいなイメージかも？

### 実装

[1293. Shortest Path in a Grid with Obstacles Elimination](https://leetcode.com/problems/shortest-path-in-a-grid-with-obstacles-elimination)
     
二次元grid(0=road, 1=obstacle)を考える。
最大k個までブロックを壊していいとき、`(0,0) -> (len(gird)-1, len(grid[0])-1)` までの最短経路を求めよ。

In [ ]:
class Solution:
    def shortestPath(self, grid: List[List[int]], k: int) -> int:
        self.grid = grid
        target = (len(grid)-1, len(grid[0])-1)
        if target == (0, 0):
            return 0
        
        if k >= self.manhattan((0,0)):
            return self.manhattan((0,0))
        
        seen = set([(k, (0, 0))]) # (num_blackables, cell)
        h = [(self.manhattan((0, 0)), 0, k, (0, 0))] # f(n), steps_so_far, num_blackables, cell
        while len(h) > 0:
            _, steps, num_b, cell = heapq.heappop(h)
            
            if cell == target:               ####### 多分ここで返さないといけない!!!!!!!!!
                return steps
            
            

            for ver, hor in [(-1, 0), (0, 1), (1, 0), (0, -1)]:
                next_cell = cell[0] + ver, cell[1] + hor

                if next_cell[0] < 0 or len(grid) <= next_cell[0]:
                    continue
                if next_cell[1] < 0 or len(grid[0]) <= next_cell[1]:
                    continue
                
                
                                            #### 同じ階層のnodeは一応全部見ないといけない!!!!!
#                 if target == next_cell:    #### 多分ここで返してはいけない!!!!!!!!!
#                     return steps + 1
                next_num_b = num_b - grid[next_cell[0]][next_cell[1]]
                if next_num_b < 0:
                    continue
                if (next_num_b, next_cell) in seen:
                    continue
                
                if self.manhattan(next_cell) <= next_num_b:
                    return steps + 1 + self.manhattan(next_cell)
                
                heapq.heappush(h, (steps+1 + self.manhattan(next_cell), steps+1, next_num_b, next_cell))
                seen.add((next_num_b, next_cell))
        return -1
    
    # ヒューリスティック関数として使っている。
    def manhattan(self, cell):
        return len(self.grid)-1 - cell[0] + len(self.grid[0])-1 - cell[1]

In [ ]:
！